### Add all imports


In [1]:
from utils import *
import re
from pyarabic.araby import strip_diacritics


### Read Data

In [2]:
training_dataset = read_training_dataset()
dev_dataset = read_dev_dataset()
# test_dataset = read_test_dataset()

letters, diacritics2id = get_letters() , get_diacritics2id()

Read training set successfully
Read validation set successfully


### Pre-process and clean data

In [3]:
# 1- Clean the data

# This pattern keeps Arabic letters, diacritics, and whitespaces and endlines
pattern = re.compile(r'[^\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF\s,.؟،;:!?\-\'"]')

# Replace unmatched characters with an empty string
cleaned_corpus = [re.sub(pattern, "", t) for t in training_dataset]
cleaned_corpus = [re.sub("\s\s+", " ", c) for c in cleaned_corpus]

# print(len(cleaned_corpus))

data,labels = [],[]

for c in cleaned_corpus:
    sentences = re.split(r"[,.؟،;:!?']+", c)  # split on all punctuation
    labels += sentences

    without_dialects = [
        strip_diacritics(s) for s in sentences
    ]  # get the letters without dialects
    data += without_dialects


# remove any spaces from line
data = [d.strip() for d in data]
labels = [l.strip() for l in labels]

# remove empty lines
data = [i for i in data if i]
labels = [i for i in labels if i]

# write the clean corpora to file
with open("./output_data/cleaned_corpus.txt", "w", encoding="utf-8") as f:
    for l in cleaned_corpus:
        f.write(l + "\n")

with open("./output_data/training_data.txt", "w", encoding="utf-8") as f:
    for d in data:
        f.write(str(d) + "\n")

with open("./output_data/labeled/training_labels.txt", "w", encoding="utf-8") as f:
    for l in labels:
        f.write(str(l) + "\n")

In [4]:
# 2- Tokenize to vocab and words
vocab = set()
tokenized_data = []
vocab, tokenized_word, tokenized_letter = tokenize_to_vocab(labels, vocab)
stemmedVocab = []


with open("./output_data/vocab.txt", "w", encoding="utf-8") as f:
    for v in vocab:
        f.write(str(v) + "\n")

with open("./output_data/tokenized_data_sentences.txt", "w", encoding="utf-8") as f:
    for ts in tokenized_word:
        f.write(str(ts) + "\n")

# tokenized_letters = tokenize_to_letters(tokenized_data)
with open("./output_data/tokenized_data_letters.txt", "w", encoding="utf-8") as f:
    for tl in tokenized_letter:
        f.write(str(tl) + "\n")
    




### Features & embeddings

### Prediction and Model